In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
import pandas as pd

from bokeh.plotting import figure, show
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, TapTool
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.sampledata.periodic_table import elements
from bokeh.transform import dodge, factor_cmap

from bokeh.io import output_file, show, curdoc
from bokeh.models.tools import HoverTool
from bokeh.models.ranges import DataRange1d
from bokeh.layouts import row, column
from bokeh.sampledata.sample_geojson import geojson
from bokeh.models import (GeoJSONDataSource, ColumnDataSource, Grid, 
                          LinearAxis, MultiPolygons, CheckboxButtonGroup, CustomJS,
                          Button, Div, Row, ImageURL, RadioButtonGroup, Dropdown,
                          Title)

from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.palettes import Category20c
from bokeh.transform import cumsum
import cmasher as cmr
from bokeh.transform import transform
from bokeh.plotting import figure, output_file, show

In [ ]:

output_file("cm-test.html")

map_labels = ['SNIa', 'SNIa-91bg', 'SNIax', 'SN-II', 'SN-Ibc', 
           'SLSN-I', 'TDE', 'KN', 'AGN', 'RRlyrae', 
           'Mdwarf', 'EBE', 'Mira', 'nuLens-Single', 'Other']

cm_kb = pd.read_fwf(r'cm_KB.txt', header=None)
cm_kb.columns = map_labels
cm_kb.index = map_labels
# this is to replace zeros with NaNs, check how it works (it does)
#cols = map_labels
#cm_kb[cols] = cm_kb[cols].replace({0:np.nan})
cm_kb.rename_axis('y_labels').rename_axis('x_labels', axis='columns')

df_stacked = pd.DataFrame(cm_kb.stack(), columns=['value']).reset_index()
df_stacked = round(df_stacked, 2)

unique_vals = df_stacked['value'].nunique() 

colors = cmr.take_cmap_colors('cmr.gothic', 
                              unique_vals, 
                              cmap_range=(1, 0.1), 
                              return_fmt='hex')

mapper = LinearColorMapper(palette=colors)

source = ColumnDataSource(df_stacked)

text_props = {"source": source, "text_align": "center", "text_baseline": "middle"}

p1 = figure(plot_width=650,
            plot_height=650, 
            title='test',
            x_range=list(cm_kb.index),
            y_range=list(reversed(cm_kb.columns)),
            toolbar_location=None, tools='',
            x_axis_location='below')

p1.xgrid.visible = False
p1.ygrid.visible = False

p1.rect(x='level_1', 
          y='level_0', 
          width=1, 
          height=1, 
          source=source,
          line_color='gray', 
          fill_color={'field': 'value', 'transform': mapper}, 
          alpha=0.9, 
          line_width=0.5)

r = p1.text(
    x='level_1',
    y='level_0',
    text='value',
    **text_props)

r.glyph.text_font_size="10px"
r.glyph.text_color='#ff0000'

color_bar = ColorBar(color_mapper=mapper, label_standoff=5, location=(500,50))

p1.axis.axis_line_color = None
#p1.axis.major_tick_line_color = None
p1.axis.major_label_text_font_size = "12px"
p1.axis.major_label_standoff = 0
p1.xaxis.major_label_orientation = 1.0

#p1.add_layout(color_bar)


p2 = figure(plot_width=600,
            plot_height=600, 
            title='test',
            x_range=list(cm_kb.index),
            y_range=list(reversed(cm_kb.columns)),
            toolbar_location=None, tools='',
            x_axis_location='below')

p2.xgrid.visible = False
p2.ygrid.visible = False


p = row(p1, p2)
show(p1)